In [ ]:
# تثبيت المكتبات المطلوبة
%pip install pytorch-forecasting pytorch-lightning pandas numpy scikit-learn matplotlib seaborn

In [4]:
# استيراد المكتبات الأساسية
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# مكتبات PyTorch
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# مكتبات PyTorch Forecasting
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss, MAE, RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# تعيين النمط للرسومات
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')

# تعيين بذرة عشوائية للنتائج المتكررة
pl.seed_everything(42)

2025-03-09 23:03:38.352828: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Global seed set to 42


42

In [6]:
# تحميل ملف البيانات
df = pd.read_csv('df_5m.csv')

# عرض المعلومات الأساسية للبيانات
print(f"شكل البيانات: {df.shape}")
print(f"الفترة الزمنية: من {df['time'].min()} إلى {df['time'].max()}")
df.head()

شكل البيانات: (224524, 32)
الفترة الزمنية: من 2022-02-21 00:15:00+00:00 إلى 2025-02-14 21:55:00+00:00


,time,Open_first,High_max,Low_min,Close_last,Volume_sum,Open_ask_first,High_ask_max,Low_ask_min,Close_ask_last,...,mid_price_mean,rolling_return_5_last,rolling_range_std_5_last,volume_zscore_mean,roc_3_last,day_of_week_first,hour_first,minute_in_hour_first,hour_sin_mean,hour_cos_mean
0,2022-02-21 00:15:00+00:00,1.35925,1.35940,1.35924,1.35940,142.35,1.35933,1.35951,1.35933,1.35950,...,1.359371,0.000026,0.000045,-0.525209,0.000081,0.0,0.0,15.0,0.0,1.0
1,2022-02-21 00:20:00+00:00,1.35940,1.35944,1.35921,1.35924,148.39,1.35949,1.35955,1.35930,1.35933,...,1.359379,-0.000016,0.000012,-0.147163,-0.000132,0.0,0.0,20.0,0.0,1.0
2,2022-02-21 00:25:00+00:00,1.35923,1.35931,1.35900,1.35909,153.99,1.35932,1.35941,1.35911,1.35921,...,1.359238,-0.000019,0.000043,0.194827,-0.000154,0.0,0.0,25.0,0.0,1.0
3,2022-02-21 00:30:00+00:00,1.35908,1.35941,1.35905,1.35935,86.37,1.35920,1.35951,1.35916,1.35949,...,1.359275,0.000035,0.000018,-0.775723,0.000184,0.0,0.0,30.0,0.0,1.0
4,2022-02-21 00:35:00+00:00,1.35938,1.35941,1.35929,1.35935,82.90,1.35949,1.35951,1.35939,1.35944,...,1.359387,-0.000009,0.000029,-0.638734,0.000022,0.0,0.0,35.0,0.0,1.0


In [7]:
# التأكد من أن عمود التاريخ هو من نوع datetime
df['time'] = pd.to_datetime(df['time'])

# ترتيب البيانات حسب التاريخ
df = df.sort_values('time')

# إضافة عمود للوقت المتسلسل (مطلوب للنموذج)
df['time_idx'] = np.arange(len(df))

# إضافة معرف المجموعة (نستخدم قيمة ثابتة لأننا نتعامل مع سلسلة زمنية واحدة)
df['group'] = 'GBP_USD'

# التحقق من القيم المفقودة
missing_values = df.isnull().sum()
print("القيم المفقودة في كل عمود:")
print(missing_values[missing_values > 0])

# معالجة القيم المفقودة إذا وجدت
if missing_values.sum() > 0:
    df = df.fillna(method='ffill')  # ملء القيم المفقودة بالقيمة السابقة
    print("تم معالجة القيم المفقودة")

القيم المفقودة في كل عمود:
Series([], dtype: int64)


In [14]:
# تحديد المتغير المستهدف (سعر الإغلاق المستقبلي)
target = 'Close_last'

# تحديد المتغيرات المستقلة (المؤشرات الفنية والأسعار)
# استخراج جميع الأعمدة باستثناء بعض الأعمدة الخاصة
excluded_columns = ['time', 'group', 'time_idx']
feature_columns = [col for col in df.columns if col != target and col not in excluded_columns]

# عرض المتغيرات المستخدمة
print(f"المتغير المستهدف: {target}")
print(f"عدد المتغيرات المستقلة: {len(feature_columns)}")
print("أول 10 متغيرات مستقلة:")
print(feature_columns[:10])

المتغير المستهدف: Close_last
عدد المتغيرات المستقلة: 30
أول 10 متغيرات مستقلة:
['Open_first', 'High_max', 'Low_min', 'Volume_sum', 'Open_ask_first', 'High_ask_max', 'Low_ask_min', 'Close_ask_last', 'Volume_ask_sum', 'minute_trend_sum']


In [15]:
# تحديد نقاط التقسيم (70% تدريب، 15% تحقق، 15% اختبار)
train_end = int(len(df) * 0.7)
val_end = int(len(df) * 0.85)

# تقسيم البيانات
training_data = df[:train_end]
validation_data = df[train_end:val_end]
test_data = df[val_end:]

print(f"بيانات التدريب: {len(training_data)} صف")
print(f"بيانات التحقق: {len(validation_data)} صف")
print(f"بيانات الاختبار: {len(test_data)} صف")

بيانات التدريب: 157166 صف
بيانات التحقق: 33679 صف
بيانات الاختبار: 33679 صف


In [16]:
# تحديد المعلمات الأساسية
max_prediction_length = 3  # عدد الخطوات الزمنية للتنبؤ (15 دقيقة)
max_encoder_length = 288   # طول السياق (24 ساعة)
batch_size = 128           # حجم الدفعة

# تكوين مجموعة بيانات التدريب
training_dataset = TimeSeriesDataSet(
    data=training_data,
    time_idx="time_idx",
    target=target,
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=[],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[target] + feature_columns,
    target_normalizer=GroupNormalizer(
        groups=["group"], transformation="softplus"
    ),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# إنشاء مجموعة بيانات التحقق باستخدام نفس الإعدادات
validation_dataset = TimeSeriesDataSet.from_dataset(
    training_dataset, validation_data, predict=True, stop_randomization=True
)

# إنشاء مجموعة بيانات الاختبار
test_dataset = TimeSeriesDataSet.from_dataset(
    training_dataset, test_data, predict=True, stop_randomization=True
)

# إنشاء محملات البيانات
train_dataloader = training_dataset.to_dataloader(
    train=True, batch_size=batch_size, num_workers=2
)
val_dataloader = validation_dataset.to_dataloader(
    train=False, batch_size=batch_size, num_workers=2
)
test_dataloader = test_dataset.to_dataloader(
    train=False, batch_size=batch_size, num_workers=2
)

# التحقق من بيانات التدريب
x, y = next(iter(train_dataloader))
print(f"شكل مدخلات الشبكة: {x['encoder_cont'].shape}")
print(f"شكل مخرجات الشبكة: {y[0].shape}")

شكل مدخلات الشبكة: torch.Size([128, 288, 35])
شكل مخرجات الشبكة: torch.Size([128, 3])


In [17]:
# تحديد مقياس الخسارة
loss = QuantileLoss()

# إنشاء نموذج TFT
tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=0.001,
    hidden_size=128,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=64,
    loss=loss,
    reduce_on_plateau_patience=5,
)

print(f"عدد معلمات النموذج: {tft.size()/1e3:.1f}k")

عدد معلمات النموذج: 1820.1k


In [ ]:
# تكوين المدرب
max_epochs = 30

# تكوين رد الاتصال للتوقف المبكر
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4,
    patience=5,
    verbose=True,
    mode="min"
)

# تكوين رد اتصال لحفظ أفضل نموذج
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="checkpoints/",
    filename="tft-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    mode="min",
)

# تكوين المسجل
logger = TensorBoardLogger("lightning_logs/", name="forex_tft")

# إنشاء المدرب
trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="auto",  # استخدام GPU إذا كان متاحًا
    callbacks=[early_stop_callback, checkpoint_callback],
    logger=logger,
    gradient_clip_val=0.1,
)

# تدريب النموذج
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# تحميل أفضل نموذج
best_model_path = checkpoint_callback.best_model_path
print(f"أفضل نموذج: {best_model_path}")
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/forex_tft

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 4.5 K 
4  | static_variable_selection          | VariableSelectionNetwork        | 77.8 K
5  | encoder_variable_selection         | VariableSelectionNetwork        | 896 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 25.7 K
7  | static_context_variable_selection  | GatedResi

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]